In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm


In [ ]:


# 1. CONFIGURATION GLOBALE
class Config:
    """Configuration du modèle et des chemins."""
    # Modèle BERT-based adapté au calcul de similarité sémantique
    MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2' 
    CSV_PATH = 'IMDb_clean.csv'
    BATCH_SIZE = 32  # Taille du lot pour l'encodage (optimisation mémoire)
    MAX_LENGTH = 128 # Longueur maximale de la séquence de tokens

# Variables globales pour la persistance en mémoire
df_movies = None
movie_title_to_index = None
movie_vectors_array = None 
# La matrice N x N de similarité n'est plus nécessaire car on la calcule 
# à la volée entre le vecteur utilisateur (1 x D) et les vecteurs bruts (N x D).

# 2. FONCTIONS D'UTILITAIRES ET DE PRÉPARATION

def mean_pooling(model_output, attention_mask):
    """
    Aggrège les vecteurs de tokens en un seul vecteur de phrase
    par la moyenne pondérée (manuel - l'étape "Option B").
    """
    # Récupère les embeddings bruts de tous les tokens
    token_embeddings = model_output.last_hidden_state
    
    # Étend le masque d'attention pour correspondre à la taille des embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    
    # Somme les embeddings (en multipliant par le masque, les zéros sont ignorés)
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    
    # Somme le masque (compte le nombre de tokens valides)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    # Retourne la moyenne
    return sum_embeddings / sum_mask

def prepare_feature_soup(df):
    """
    Nettoie les données et crée la chaîne de caractères structurée ('soup') 
    pour le modèle Transformer.
    """
    print(" Préparation de la Feature Soup...")
    
    # 1. Gestion des valeurs manquantes et nettoyage de base
    df['description'] = df['description'].fillna('')
    df['side_genre'] = df['side_genre'].fillna('')
    df['Director'] = df['Director'].fillna('Unknown')
    
    # Nettoyage des espaces excédentaires
    for col in ['main_genre', 'side_genre', 'Actors', 'Movie_Title', 'Director']:
        df[col] = df[col].astype(str).str.strip()
    
    # 2. Création de la "Soup" structurée pour le modèle
    df['soup'] = (
        "Title: " + df['Movie_Title'] + ". " +
        "Director: " + df['Director'] + ". " +
        "Genres: " + df['main_genre'] + ", " + df['side_genre'] + ". " +
        "Stars: " + df['Actors'] + ". " +
        "Plot: " + df['description']
    )
    
    # Nettoyage final de la string
    df['soup'] = df['soup'].str.replace(r'\s+', ' ', regex=True).str.strip()
    
    print(f"✅ Soup créée pour {len(df)} films.")
    return df

# 3. INITIALISATION (La partie LOURDE - Exécutée une seule fois)

def initialize_and_encode():
    """
    Exécute les étapes lourdes (encodage) et stocke les résultats 
    dans les variables globales.
    """
    global df_movies, movie_title_to_index, movie_vectors_array
    
    if movie_vectors_array is not None:
        print("✅ Vecteurs déjà encodés en mémoire. Saut de l'étape lourde.")
        return

    print("\n---  Démarrage de l'Encodage LOURD ---")
    
    # A. Chargement et Préparation
    df = pd.read_csv(Config.CSV_PATH)
    df_movies = prepare_feature_soup(df.copy())
    
    # Crée le mapping Titre -> Index (clé essentielle)
    movie_title_to_index = pd.Series(df_movies.index, index=df_movies['Movie_Title']).drop_duplicates()
    
    # B. Configuration du Modèle et de l'Environnement
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f" Utilisation du périphérique: {device}")
    
    tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
    model = AutoModel.from_pretrained(Config.MODEL_NAME).to(device).eval() # Mode évaluation
    
    all_embeddings = []
    texts = df_movies['soup'].tolist()

    # C. Boucle d'Encodage
    for i in tqdm(range(0, len(texts), Config.BATCH_SIZE), desc="Génération des Embeddings"):
        batch_texts = texts[i : i + Config.BATCH_SIZE]
        
        # 1. Tokenization (Hugging Face)
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, 
                                  max_length=Config.MAX_LENGTH, return_tensors='pt').to(device)
        
        # 2. Modèle Inference (PyTorch / Transformer)
        with torch.no_grad():
            model_output = model(**encoded_input)
            
        # 3. Pooling (Agrégation manuelle) et Normalisation
        batch_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
        
        all_embeddings.append(batch_embeddings.cpu().numpy())

    movie_vectors_array = np.vstack(all_embeddings)
    print(f"✅ Matrice de vecteurs bruts générée: {movie_vectors_array.shape}")
    print("--- 🏁 Initialisation Terminée ---")




--- 🧠 Démarrage de l'Encodage LOURD ---
🍜 Préparation de la Feature Soup...
✅ Soup créée pour 5449 films.
⚙️ Utilisation du périphérique: cpu


Génération des Embeddings: 100%|██████████| 171/171 [02:40<00:00,  1.06it/s]

✅ Matrice de vecteurs bruts générée: (5449, 384)
--- 🏁 Initialisation Terminée ---

--- Recommandations pour la sélection de 5 films (Top 5) ---
Score      | Movie Title
----------------------------------------
0.6923     | xXx
0.6906     | Tenet
0.6884     | 6 Underground
0.6831     | John Wick: Chapter 2
0.6807     | 8MM

--- Recommandations pour la sélection de 5 films (Top 5) ---
Score      | Movie Title
----------------------------------------
0.6686     | Zookeeper
0.6615     | Jumanji
0.6599     | The Wild
0.6544     | Wildlife
0.6432     | Toy Story 2

--- Recommandations pour la sélection de 5 films (Top 5) ---
Score      | Movie Title
----------------------------------------
0.6426     | Click
0.6373     | The International
0.6329     | The Contract
0.6272     | Maps to the Stars
0.6230     | Spartacus


In [ ]:

# 4. LOGIQUE DE RECOMMANDATION (La partie RAPIDE)

def get_recommendations(movie_titles: list, top_n=5):
    """
    Calcule le vecteur de profil utilisateur à partir d'une liste de films
    et trouve les top N films les plus similaires.
    """
    global df_movies, movie_title_to_index, movie_vectors_array
    
    if movie_vectors_array is None:
        print(" ERREUR : Le modèle n'a pas été initialisé. Exécutez initialize_and_encode() d'abord.")
        return []

    # 1. Récupérer les index des films sélectionnés
    try:
        indices_selected = [movie_title_to_index[title] for title in movie_titles]
    except KeyError as e:
        print(f"Film non trouvé dans la base de données: {e}")
        return []

    # 2. Créer le Vecteur de Profil Utilisateur (Agrégation)
    selected_vectors = movie_vectors_array[indices_selected]
    user_profile_vector = np.mean(selected_vectors, axis=0, keepdims=True)
    
    # 3. Comparaison (Cosine Similarity)
    # Comparer le vecteur profil (1 x D) à tous les films (N x D)
    scores_vector = cosine_similarity(user_profile_vector, movie_vectors_array)
    
    # 4. Trier et Filtrer
    scores = list(enumerate(scores_vector[0])) 
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
    # Indices des films déjà sélectionnés pour l'exclusion
    selected_indices_set = set(indices_selected)
    recommendations_list = []
    
    for idx, score in sorted_scores:
        if idx not in selected_indices_set:
            # Ajouter le titre et le score
            recommendations_list.append({
                'title': df_movies.iloc[idx]['Movie_Title'],
                'score': score
            })
        
        if len(recommendations_list) >= top_n:
            break

    # 5. Affichage et retour
    print(f"\n--- Recommandations pour la sélection de {len(movie_titles)} films (Top {top_n}) ---")
    print(f"{'Score':<10} | {'Movie Title'}")
    print("-" * 40)
    for rec in recommendations_list:
        print(f"{rec['score']:.4f}     | {rec['title']}")
        
    return recommendations_list


# 5. EXÉCUTION POUR TESTER ET VÉRIFIER
if __name__ == "__main__":
    
    # --- ÉTAPE 1: Initialisation (Executez une seule fois) ---
    # Cette étape est longue car elle télécharge le modèle et calcule les 5449 vecteurs.
    initialize_and_encode() 

    # --- ÉTAPE 2: Tests Répétitifs (Instantané) ---

    # Exemple 1: Profil Action/Thriller (The Dark Knight, Inception sont de bons exemples)
    action_thriller_selection = [
        'The Dark Knight', 
        'Inception', 
        '1 - Nenokkadine',
        'Mission: Impossible - Fallout',
        'Mad Max: Fury Road'
    ]
    get_recommendations(action_thriller_selection, top_n=5)

    # Exemple 2: Profil Animation/Famille
    animation_selection = [
        'Zootopia', 
        'The Lion King', 
        'Finding Nemo', 
        'Toy Story',
        'Inside Out'
    ]
    get_recommendations(animation_selection, top_n=5)
    
    # Exemple 3: Films de Drames Historiques
    drama_history_selection = [
        'Schindler\'s List',
        'The Pianist',
        '12 Years a Slave',
        'Lincoln',
        'A Beautiful Mind'
    ]
    get_recommendations(drama_history_selection, top_n=5)